### HunyuanVideoPipiline类

其`__call__`方法主要有以下流程:

(1)check_inputs检查输入

(2)准备输入

- text_embeddings(使用encode_prompt())
- timestep(使用retrieve_timesteps)
- latents(使用prepare_latents)

(3)去噪(denoising_step循环)

(4)解码输出

1. text_embeddings

有`prompt_embeds, negative_prompt_embeds, prompt_mask, negative_prompt_mask`作为输出。

如果使用CFG，则会将其concat成`[prompt_embeds, negative_prompt_embeds]`,mask同理

2. timestep

输出递降的`<list>` `timesteps`和`<int>` `num_inference_steps`

3. latents

会先将`video_length`做$(L-1)//4+1$，接着将`video_length = 33` `num_channels_latents = 16`送入`prepare_latents()`.在该函数中，若
- 启用`latent_concat`(默认关闭),则会生成`[B,C,F,H,W]=[1,7,33,24,42]`的`latents`
- 启用`i2v_stability`(默认开启),则会线性注入`0.001`的`img_latents`,得到`[B,C,F,H,W]=[1,16,33,24,42]`的`latents`
- 启用`FlowMatching`调度器且有`init_noise_sigma`属性，则会把`latents`的方差变为$init\_noise\_sigma^2 * I$

如果启用`latent_concat`，`img_latents_concat`会是形如`[1,16,33,24,42]`,第一帧存在，后续帧全为0的`<tensor>`,`i2v_mask`是第一元1，后续0的`[33,]`张量，`mask_concat`是形如`[1,1,33,24,42]`，第一帧全为1,后续帧全为0的张量.

4. denoising step

使用`for i, t in enumerate(timesteps)`循环
- 若启用`latent_concat`,`latent_model_input`会是`latents, img_latents_concat, mask_concat`按**通道**维拼接的结果，形如`[1,7+16+1,33,24,42]`,否则为`[1,16,33,24,42]`的`latents`
- 若启用`token_replace`,在去噪前，每个时间步的第一帧都将被`img_latents`替换，去噪只在后续帧执行，去噪后会再与`img_latents`拼接。这一重复可能是为了便于回调。

5. decode

$latents = latents/vae\_sacling\_factor+vae\_shifting\_factor$,

接着将其通过VAE，得到`image`。将其从`[-1,1]`转换到`(0,1)`,即作为`HunyuanVideoPipelineOutput`的属性.

### HunyuanVideoSampler类

其`predict`方法主要实现以下功能，并最后输出一个`out_dict`：

(1)初始化随机种子和FlowMatchDiscreteScheduler

(2)根据图片高宽比选择适配的视频尺寸,并裁剪图片

(3)调用pipeline完成inference

注：参数ulysses或ring会启用并行VAE编码

1. seeds

将不同类型的种子输入统一为`batch_size * num_videos_per_prompt`个种子的`<list>` `seeds`

2. crop

根据所选清晰度，依据最邻近高宽比得到和原图片尺寸最匹配的`closest_size`和`closest_ratio`，用于裁剪。接着`[3,H,W]`的RGB图像形状会被缩放、裁剪，接着值被缩放到`[-0.5,0.5]`, unsqueeze维度到`[1,3,1,H,W]`,

接着，经VAE编码，再乘系数$vae\_scaling\_factor$,得到`img_latents`.图片可经`<list[uint8]>` `semantic_images`传入，或只使用图片路径导入.

### VAE

其中`EncoderCausal3D`和`DecoderCausal3D`分别实现VAE的主干部分，`AutoencoderKLCausal3D`类的`encode`和`_decode`方法调用这两个神经网络.tiling,slicing,blend,fuseqkv,attn是几个额外功能,其中只有`vae_tiling=True`,故只考虑这部分.

"the tiles overlap and are blended together to form a smooth output"太工程了！

1. spatial_tiled_encode

它会把图像分割成小块，装入二元列表`rows`中，分别编码，然后拼接. 其中有关键参数
- `tile_sample_min_size`: 默认为32,将图像按空间维度裁剪为方体，但最后一行/列不方
- `tile_overlap_factor`: 重叠比例，默认为0.25
- `overlap_size`: 相邻 tile 起点之间的步长
- `blend_extent`: 需要融合的宽度/高度
- `row_limit`: 切片边缘,使用时为`[:,:,:,:row_limit,:row_limit]`. 每次融合其左/上方的图片，并在存储时去掉其右侧、下侧`blend_extent`宽度的像素


In [ ]:
2. temporal_tiled_encode

它会按时间维度将图像分成小块，并在每个时间块递归地调用`spatial_tiled_encode()`, 待空间编码融合后, 再进行时间编码、融合